In [1]:
import numpy as np
import pandas as pd
from gates import Network
from sklearn.preprocessing import OneHotEncoder

from optimizers import GeneticOptimizer
from optimizers.genetic import RouletteWheelSelection, RankSelection, NRandomChangesMutation, MultiPointCrossing, HighestAccuracyFitness, LowestMseFitness

## Preparing data

In [2]:
data = pd.read_csv('data/zoo.csv')

In [3]:
data

,animal_name,hair,feathers,eggs,milk,airborne,aquatic,predator,toothed,backbone,breathes,venomous,fins,legs,tail,domestic,catsize,class_type
0,aardvark,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
1,antelope,1,0,0,1,0,0,0,1,1,1,0,0,4,1,0,1,1
2,bass,0,0,1,0,0,1,1,1,1,0,0,1,0,1,0,0,4
3,bear,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
4,boar,1,0,0,1,0,0,1,1,1,1,0,0,4,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,wallaby,1,0,0,1,0,0,0,1,1,1,0,0,2,1,0,1,1
97,wasp,1,0,1,0,1,0,0,0,0,1,1,0,6,0,0,0,6
98,wolf,1,0,0,1,0,0,1,1,1,1,0,0,4,1,0,1,1
99,worm,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,7


In [4]:
legs = data['legs']
other_features = data.drop(columns=['animal_name', 'legs', 'class_type']).to_numpy()
legs_onehot = OneHotEncoder().fit_transform(legs.to_numpy().reshape(-1, 1)).todense()
features = np.hstack([other_features, legs_onehot])
features = np.array(features.astype(bool))

In [5]:
y = data['class_type'].to_numpy()

is_mammal = y == 1
is_bird = y == 2
is_reptile = y == 3
is_fish = y == 4
is_amphibian = y == 5
is_bug = y == 6

In [6]:
features

array([[ True, False, False, ..., False, False, False],
       [ True, False, False, ..., False, False, False],
       [False, False,  True, ..., False, False, False],
       ...,
       [ True, False, False, ..., False, False, False],
       [False, False,  True, ..., False, False, False],
       [False,  True,  True, ..., False, False, False]])

## Optimizing network

In [7]:
outputs = is_mammal.reshape(-1, 1)
network = Network(21, [40, 50, 50, 40, 1])
print(f'Accuracy: {network.evaluate(features, outputs)}')

Accuracy: (0.594059405940594, 0.40594059405940597)


In [8]:
optimizer = GeneticOptimizer(
    fitness=LowestMseFitness(),
    crossing=MultiPointCrossing(3),
    mutation=NRandomChangesMutation(4),
    selection=RouletteWheelSelection(),
    cross_prob=0.7,
    mutate_prob=0.2,
    n_chromosomes=100,
    n_generations=200,
    n_elite=4,
    patience=50,
)

optimizer.optimize(network, features, outputs)
loss, accuracy = network.evaluate(features, outputs)
print(f'Accuracy: {accuracy}')

Optimizing 905 parameters


Evolution:   5%|▌         | 10/200 [00:07<02:31,  1.25it/s, fitness: inf, mse: 0.000, accuracy: 1.000]              

Accuracy: 1.0
